In [11]:
import psycopg2
from psycopg2.extras import RealDictCursor
import os
# Use DATABASE_URL_UNPOOLED for direct connection
conn = psycopg2.connect(os.getenv('DATABASE_URL_UNPOOLED'))
cur = conn.cursor()
q = """SELECT * FROM users"""
cur.execute(q)
results = cur.fetchall()
print(results)
conn.close()

[('adamgent5', 'a-dog', 'adam.gent5@me.com', {'scope': 'playlist-modify-private playlist-modify-public user-read-email user-read-private', 'expires_in': 3600, 'token_type': 'Bearer', 'access_token': 'BQB7YRCspwkaf9M8NHH_CUzsYGUGRLXEElSRf5SxtBlnl-YRi8kM9t1iegsU8jFyCX-pUBsEP3FU0T4ph5jFjTcii5OuYIMFHrZeBAhEIaelfMqA3JQNARtarUTKk7dJO0YsC0Q8O_1SYuHs_AZx41jiaN3EwjjgLOpw_uutC4i466bKgdIx2Un_M5oIU9IeCYLu-aJyrmLsIrB99MyiW4qsyIFH5coB2qesGH4tEWgnEPRUZXX2WTn1341ug7H37b3j1dUp-aF6k90bdYyztmz9jkLKtNhPpaU', 'refresh_token': 'AQATwwUQwFPUCOUtTNLOFDh_RjPjCeNQBZiKm7sokCjNlsX10l8dNljI_ttDFP3I4LsJLCMPqBfP_FGXY-BRJ2BXCPnrvqi7gSPINStrx1WZ90h4XgoKw3bBYJnK0wqUqY4'}, True, datetime.datetime(2025, 9, 25, 21, 8, 20, 819254), datetime.datetime(2025, 9, 27, 4, 55, 20, 763400)), ('meekdylan', 'meekdylan', 'dmeek453@gmail.com', {'scope': 'playlist-modify-private playlist-modify-public user-read-email user-read-private', 'expires_in': 3600, 'token_type': 'Bearer', 'access_token': 'BQByfZUpa2iSf-1IhibafHz0tbqvgTP6LMoVwAac

In [4]:

try:
    # Create cursor from your connection
    cur = conn.cursor()
    # If you need to modify the schema later, use ALTER TABLE or migrations
    create_table_query = """
    CREATE TABLE venues (
        venue_id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        address VARCHAR(500),
        city VARCHAR(100) NOT NULL,
        url VARCHAR(500) NOT NULL UNIQUE,
        scraping_config JSONB NOT NULL,
        is_active BOOLEAN DEFAULT TRUE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """

    cur.execute(create_table_query)

    # Create index on city for faster queries
    cur.execute("CREATE INDEX idx_venues_city ON venues(city);")

    # Create index on is_active for filtering
    cur.execute("CREATE INDEX idx_venues_active ON venues(is_active);")

    conn.commit()
    print("✅ Venues table created successfully!")
finally:
    conn.close()

✅ Venues table created successfully!


In [10]:
# Delete all venues so you can re-run the insert
cur = conn.cursor()

#cur.execute("DELETE FROM venues;")

c#onn.commit()
#print("✅ All venues deleted!")

✅ All venues deleted!


In [11]:
import json

# Define the three venues with their scraping configs
venues_data = [
    {
        "name": "The Chapel",
        "address": "777 Valencia St",
        "city": "San Francisco",
        "url": "https://thechapelsf.com/music/?list1page=1",
        "scraping_config": {
            "base_url": "https://thechapelsf.com/music/?list1page=1",
            "pagination": {
                "enabled": True,
                "pages": 2,
                "url_pattern": "https://thechapelsf.com/music/?list1page={page}"
            },
            "selectors": {
                "event_container": "div.event-info-block",
                "artist": "p.fs-12.headliners",
                "date": "p.fs-18.bold.mt-1r.date",
                "genre": "p.fs-12.genre",
                "cancellation_indicator": None
            },
            "date_format": "%a %b %d",
            "filters": {
                "check_cancelled": False
            }
        }
    },
    {
        "name": "The Independent",
        "address": "628 Divisadero St",
        "city": "San Francisco",
        "url": "https://www.theindependentsf.com/",
        "scraping_config": {
            "base_url": "https://www.theindependentsf.com/",
            "pagination": {
                "enabled": False
            },
            "selectors": {
                "event_container": "div.tw-section",
                "artist": "div.tw-name > a",
                "date": "span.tw-event-date",
                "genre": None,
                "cancellation_indicator": "div.tw-info-price-buy-tix > a:nth-of-type(2)"
            },
            "date_format": "%m.%d",
            "filters": {
                "check_cancelled": True,
                "cancelled_text": "Cancelled"
            }
        }
    },
    {
        "name": "Rickshaw Stop",
        "address": "155 Fell St",
        "city": "San Francisco",
        "url": "https://rickshawstop.com/?list1page=1",
        "scraping_config": {
            "base_url": "https://rickshawstop.com/?list1page=1",
            "pagination": {
                "enabled": True,
                "pages": 2,
                "url_pattern": "https://rickshawstop.com/?list1page={page}"
            },
            "selectors": {
                "event_container": "div.event-info-block",
                "artist": "p.fs-12.headliners",
                "date": "p.fs-18.bold.mt-1r.date",
                "genre": None,
                "cancellation_indicator": None
            },
            "date_format": "%a %b %d",
            "filters": {
                "exclude_genres": [],
                "check_cancelled": False
            }
        }
    }
]

# Insert each venue
cur = conn.cursor()

insert_query = """
INSERT INTO venues (name, address, city, url, scraping_config)
VALUES (%s, %s, %s, %s, %s)
RETURNING venue_id, name;
"""

for venue in venues_data:
    cur.execute(
        insert_query,
        (
            venue["name"],
            venue["address"],
            venue["city"],
            venue["url"],
            json.dumps(venue["scraping_config"])
        )
    )
    venue_id, name = cur.fetchone()
    print(f"✅ Inserted {name} (ID: {venue_id})")

conn.commit()
print("\n🎉 All venues inserted successfully!")

✅ Inserted The Chapel (ID: 7)
✅ Inserted The Independent (ID: 8)
✅ Inserted Rickshaw Stop (ID: 9)

🎉 All venues inserted successfully!


In [3]:
# Create cursor from your connection
cur = conn.cursor()

# First create users table
create_users_table = """
CREATE TABLE users (
    spotify_user_id VARCHAR(100) PRIMARY KEY,   -- Spotify's user ID
    display_name VARCHAR(255),
    email VARCHAR(255),
    spotify_credentials JSONB NOT NULL,         -- Token info, refresh tokens, etc.
    is_active BOOLEAN DEFAULT TRUE,             -- For soft deletes
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

# Then create playlists table with foreign key (no cascade)
create_playlists_table = """
CREATE TABLE playlists (
    playlist_id VARCHAR(50) PRIMARY KEY,        -- Spotify playlist ID
    spotify_user_id VARCHAR(100) REFERENCES users(spotify_user_id),
    playlist_name VARCHAR(255) NOT NULL,        -- User-customizable name
    playlist_description TEXT,                  -- User-customizable description
    preferred_venues INTEGER[] DEFAULT '{}',    -- Array of venue_ids for this playlist
    preferred_genres TEXT[] DEFAULT '{}',       -- Array of genre strings
    excluded_genres TEXT[] DEFAULT '{}',        -- Array of genres to exclude
    songs_per_artist INTEGER DEFAULT 3,        -- Preference for this playlist
    is_active BOOLEAN DEFAULT TRUE,             -- For soft deletes
    playlist_updated_at TIMESTAMP,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

cur.execute(create_users_table)
cur.execute(create_playlists_table)

# Create indexes
cur.execute("CREATE INDEX idx_playlists_spotify_user_id ON playlists(spotify_user_id);")
cur.execute("CREATE INDEX idx_playlists_preferred_venues ON playlists USING GIN(preferred_venues);")
cur.execute("CREATE INDEX idx_users_active ON users(is_active);")
cur.execute("CREATE INDEX idx_playlists_active ON playlists(is_active);")

conn.commit()
print("Users and playlists tables created successfully!")

Users and playlists tables created successfully!


In [9]:
conn = psycopg2.connect(os.getenv('DATABASE_URL_UNPOOLED'))
cur = conn.cursor()
cur.execute(
"""ALTER TABLE playlists 
ADD COLUMN days_ahead INTEGER DEFAULT 21;"""
)
# Add a check constraint to ensure reasonable values
cur.execute(
"""
ALTER TABLE playlists 
ADD CONSTRAINT check_days_ahead CHECK (days_ahead >= 1 AND days_ahead <= 365);""")
conn.commit()
cur.execute("SELECT column_name, data_type, column_default FROM information_schema.columns WHERE table_name = 'playlists';")
print(cur.fetchall())

[('songs_per_artist', 'integer', '3'), ('is_active', 'boolean', 'true'), ('playlist_updated_at', 'timestamp without time zone', None), ('created_at', 'timestamp without time zone', 'CURRENT_TIMESTAMP'), ('updated_at', 'timestamp without time zone', 'CURRENT_TIMESTAMP'), ('days_ahead', 'integer', '21'), ('preferred_venues', 'ARRAY', "'{}'::integer[]"), ('excluded_genres', 'ARRAY', "'{}'::text[]"), ('spotify_user_id', 'character varying', None), ('playlist_name', 'character varying', None), ('playlist_description', 'text', None), ('preferred_genres', 'ARRAY', "'{}'::text[]"), ('playlist_id', 'character varying', None)]
